1. CÁC BƯỚC CƠ BẢN ĐỂ XÂY DỰNG MÔ HÌNH, IMPORT CÁC THỰ VIỆN VÀ MODULE

In [ ]:
!pip install pyvi

In [ ]:
!unzip /content/drive/MyDrive/data_train.zip

In [ ]:
from pyvi import ViTokenizer, ViPosTagger
import gensim
import os
import pickle
import codecs
import re
from tqdm import tqdm
import regex
import pandas as pd
from tokenize import group
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [ ]:
!unzip /content/drive/MyDrive/data_train.zip

2. TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
def remove_html(txt):
  return regex.sub(r'<[^>]*>', '', txt)

In [ ]:
stopword = set()
with open('/content/drive/MyDrive/vietnamese-stopwords.txt', 'r', encoding='utf-8') as _fp:
  word = _fp.readlines()
stopword = [n.replace('\n', '') for n in word]
def remove_stopwords(line):
  words = []
  for word in line.strip().split():
    if word not in stopword:
      words.append(word)
  return ' '.join(words)

In [ ]:
def text_preprocess(document):
   document = remove_html(document)
   lemmatizer = WordNetLemmatizer()
   document = ' '.join([lemmatizer.lemmatize(word) for word in document.split()])
   document = ViTokenizer.tokenize(document)
   document = document.lower()
   document = regex.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', '', document)
   document = regex.sub(r'\s+', ' ', document).strip()
   document = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), document, flags=re.IGNORECASE)
   return document

3. TẢI DỮ LIỆU

In [ ]:
def load_data(directory_path, label):
    data_frames = []
    file_list = os.listdir(directory_path)
    for file_name in file_list:
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
            data = text_preprocess(data)
            data = remove_stopwords(data)
            df = pd.DataFrame({'text': [data], 'label': [label]})
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)
pos_path = '/content/data_train/train/pos'
neg_path = '/content/data_train/train/neg'
pos_data = load_data(pos_path, 1) 
neg_data = load_data(neg_path, 0)
traindata = pd.concat([pos_data, neg_data], ignore_index=True)
traindata.to_csv('/content/traindata.csv', index=False)

In [ ]:
def load_data(directory_path, label):
    data_frames = []
    file_list = os.listdir(directory_path)
    for file_name in file_list:
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
            data = text_preprocess(data)
            data = remove_stopwords(data)
            df = pd.DataFrame({'text': [data], 'label': [label]})
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)
pos_path = '/content/data_train/test/pos'
neg_path = '/content/data_train/test/neg'
pos_data = load_data(pos_path, 1)
neg_data = load_data(neg_path, 0)
testdata = pd.concat([pos_data, neg_data], ignore_index=True)
testdata.to_csv('/content/testdata.csv', index=False)

4. TRÍCH XUẤT ĐẶC TRƯNG TỪ DỮ LIỆU

In [ ]:
traindata = pd.read_csv('traindata.csv')
testdata = pd.read_csv('testdata.csv')
X_train = traindata['text']
y_train = traindata['label']
X_test = testdata['text']
y_test = testdata['label']

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_train_tfidf)
X_train_tfidf_svd = svd.transform(X_train_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

5. MÔ HÌNH

5.1 KNN

5.2 SVM

5.3 NAIVE BAYES